In [ ]:
import numpy as np
import pyvista as pv

In [ ]:
# using pyvista, generate a simple 1d unstructured grid, made of VTK_LINE
# objects, with 10 vertices and 9 edges, distributed from 0 to 1. Then add 1
# scalar point data field containing the x of each vertex, one vector field
# containing the x,y,z coordinates of the vertices, one scalar cell data field,
# containing the x coordinate of the center of the edge, and one vector cell
# data field containing the coordinates of the center of the edge

In [ ]:
# Create 10 points along x from 0 to 1, (y from 1 to 2, and z from 3 to 4)
points = np.zeros((10, 3))
points[:, 0] = np.linspace(0, 1, 10)
# points[:, 1] = np.linspace(1, 2, 10)
# points[:, 2] = np.linspace(2, 3, 10)

# Create 9 VTK_LINE cells connecting consecutive points
cells = []
for i in range(9):
    cells.extend([2, i, i + 1])  # 2 = number of points per line

cells = np.array(cells)

# Create the unstructured grid
grid = pv.UnstructuredGrid(cells, np.full(9, pv.CellType.LINE), points)

# Point data: scalar field (x), vector field (xyz)
grid.point_data["x"] = points[:, 0]
grid.point_data["xyz"] = points

# Cell data: scalar field (center x), vector field (center xyz)
cell_centers = 0.5 * (points[:-1] + points[1:])
grid.cell_data["center_x"] = cell_centers[:, 0]
grid.cell_data["center_xyz"] = cell_centers

# Save to VTK file
grid.save("data_1_1.vtk", binary=False)

In [ ]:
# using pyvista, generate a simple 2d unstructured grid of a square, made of
# VTK_TRIANGLE objects, with 8 triangles. Then add 1 scalar point data field
# containing the x of each vertex, one vector field containing the x,y,z
# coordinates of the vertices, one scalar cell data field, containing the x
# coordinate of the center of the triangle, and one vector cell data field
# containing the coordinates of the center of the triangle

In [ ]:
points2 = np.zeros((10, 3))
for row in range(2):
  for col in range(5):
    idx = row * 5 + col
    points2[idx, 0] = col / 4.0  # x
    points2[idx, 1] = row * 1.0  # y
    points2[idx, 2] = 0.0        # z

# build 8 triangles from splitting 4 quads
triangles = []
for col in range(4):
  a = col            # bottom row index
  b = col + 5        # top row same column
  c = col + 1
  d = col + 6        # top row next column
  triangles.append([a, b, d])  # triangle 1 of quad
  triangles.append([a, d, c])  # triangle 2 of quad

# build VTK cell array for the 8 triangles
tri_cells = np.array([elem for tri in triangles for elem in ([3] + tri)], dtype=np.int64)

# cell types for 8 triangle cells
cell_types = np.full(len(triangles), pv.CellType.TRIANGLE, dtype=np.uint8)

# create the unstructured grid
grid3 = pv.UnstructuredGrid(tri_cells, cell_types, points2)

# point data: scalar x and vector xyz
grid3.point_data["x"] = points2[:, 0]
grid3.point_data["xyz"] = points2

# cell centers and cell data: scalar center_x and vector center_xyz (use first 8 centers)
cell_centers = grid3.cell_centers().points
grid3.cell_data["center_x"] = cell_centers[:, 0]
grid3.cell_data["center_xyz"] = cell_centers

# save to VTK
grid3.save("data_2_2_simplex.vtk", binary=False)

In [ ]:
# using pyvista, generate a simple 2d unstructured grid of a square, made of
# VTK_QUAD objects, with 4 squares. Then add 1 scalar point data field
# containing the x of each vertex, one vector field containing the x,y,z
# coordinates of the vertices, one scalar cell data field, containing the x
# coordinate of the center of the square, and one vector cell data field
# containing the coordinates of the center of the square

In [ ]:
# build a 3x3 grid of points -> 4 quads (2x2)
points_quad = np.zeros((9, 3))
for r in range(3):
  for c in range(3):
    points_quad[r * 3 + c, 0] = c / 2.0  # x in [0,1]
    points_quad[r * 3 + c, 1] = r / 2.0  # y in [0,1]
    points_quad[r * 3 + c, 2] = 0.0

# build 4 quads (each cell: [4, i0, i1, i2, i3])
quads = []
for r in range(2):
  for c in range(2):
    bl = r * 3 + c
    br = bl + 1
    tl = (r + 1) * 3 + c
    tr = tl + 1
    quads.append([4, bl, br, tr, tl])

quads = np.array([v for cell in quads for v in cell], dtype=np.int64)
cell_types_quad = np.full(4, pv.CellType.QUAD, dtype=np.uint8)

grid_quad = pv.UnstructuredGrid(quads, cell_types_quad, points_quad)

# point data
grid_quad.point_data["x"] = points_quad[:, 0]
grid_quad.point_data["xyz"] = points_quad

# cell centers and cell data
centers = grid_quad.cell_centers().points
grid_quad.cell_data["center_x"] = centers[:, 0]
grid_quad.cell_data["center_xyz"] = centers

# save
grid_quad.save("data_2_2_quad.vtk", binary=False)

In [ ]:
# using pyvista, generate a simple 3d unstructured grid of a square, made of
# VTK_TET objects splitting 8 cubes. Then add 1 scalar point data field
# containing the x of each vertex, one vector field containing the x,y,z
# coordinates of the vertices, one scalar cell data field, containing the x
# coordinate of the center of the square, and one vector cell data field
# containing the coordinates of the center of the square

In [ ]:
# create 3x3x3 points for 2x2x2 cubes
points3d = np.zeros((27, 3))
idx = 0
for z in range(3):
  for y in range(3):
    for x in range(3):
      points3d[idx] = (x / 2.0, y / 2.0, z / 2.0)
      idx += 1

# tetra decomposition for each cube (2x2x2 -> 8 cubes)
tets = []
for z in range(2):
  for y in range(2):
    for x in range(2):
      b = z * 9 + y * 3 + x
      v0 = b
      v1 = b + 1
      v2 = b + 4  # +1 +3
      v3 = b + 3
      v4 = b + 9
      v5 = b + 10
      v6 = b + 13
      v7 = b + 12
      # 5-tet split of a cube
      tets.append([v0, v1, v3, v4])
      tets.append([v1, v2, v3, v6])
      tets.append([v1, v4, v5, v6])
      tets.append([v3, v4, v6, v7])
      tets.append([v1, v3, v4, v6])

# build VTK cell array for tets
tet_cells = np.array([elem for tet in tets for elem in ([4] + tet)], dtype=np.int64)
cell_types_tet = np.full(len(tets), pv.CellType.TETRA, dtype=np.uint8)

# create the unstructured grid
grid_tet = pv.UnstructuredGrid(tet_cells, cell_types_tet, points3d)

# point data: scalar x and vector xyz
grid_tet.point_data["x"] = points3d[:, 0]
grid_tet.point_data["xyz"] = points3d

# cell centers and cell data
centers_tet = grid_tet.cell_centers().points
grid_tet.cell_data["center_x"] = centers_tet[:, 0]
grid_tet.cell_data["center_xyz"] = centers_tet

# save to VTK
grid_tet.save("data_3_3_tets.vtk", binary=False)

In [ ]:
# using pyvista, generate a simple 3d unstructured grid of a square, made of 8
# VTK_HEX cubes. Then add 1 scalar point data field containing the x of each
# vertex, one vector field containing the x,y,z coordinates of the vertices, one
# scalar cell data field, containing the x coordinate of the center of the
# square, and one vector cell data field containing the coordinates of the
# center of the square

In [ ]:
# build 8 hexahedral cells (2x2x2 cubes) using existing points3d
hexes = []
for z in range(2):
  for y in range(2):
    for x in range(2):
      b = z * 9 + y * 3 + x
      v0 = b
      v1 = b + 1
      v2 = b + 4
      v3 = b + 3
      v4 = b + 9
      v5 = b + 10
      v6 = b + 13
      v7 = b + 12
      hexes.append([8, v0, v1, v2, v3, v4, v5, v6, v7])

# validate no repeated vertex in any cell
for cell in hexes:
    ids = cell[1:]
    if len(ids) != len(set(ids)):
        raise ValueError(f"Duplicate vertex index in cell: {cell}")

# use int32 for cell connectivity (VTK expects 32-bit indices on some builds)
hex_cells = np.array([v for cell in hexes for v in cell], dtype=np.int32)
cell_types_hex = np.full(len(hexes), pv.CellType.HEXAHEDRON, dtype=np.uint8)

grid_hex = pv.UnstructuredGrid(hex_cells, cell_types_hex, points3d)

# point data: scalar x and vector xyz
grid_hex.point_data["x"] = points3d[:, 0]
grid_hex.point_data["xyz"] = points3d

# cell centers and cell data
centers_hex = grid_hex.cell_centers().points
grid_hex.cell_data["center_x"] = centers_hex[:, 0]
grid_hex.cell_data["center_xyz"] = centers_hex

# save to VTK
grid_hex.save("data_3_3_hexes.vtk", binary=False)